In [ ]:
# Imports
import babypandas as bpd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Imports for animation
from lec18 import sampling_animation

from IPython.display import display, IFrame, HTML, YouTubeVideo

def show_permutation_testing_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vSovXDonR6EmjrT45h4pY1mwmcKFMWVSdgpbKHC5HNTm9sbG7dojvvCDEQCjuk2dk1oA4gmwMogr8ZL/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 569
    display(IFrame(src, width, height))
    
def show_bootstrapping_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vS_iYHJYXSVMMZ-YQVFwMEFR6EFN3FDSAvaMyUm-YJfLQgRMTHm3vI-wWJJ5999eFJq70nWp2hyItZg/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 509
    display(IFrame(src, width, height))

# Lecture 18 – Causality and Bootstrapping

## DSC 10, Spring 2022

### Announcements

- Homework 5 is due **tomorrow at 11:59pm**.
- Lab 6 is due **Saturday 5/14 at 11:59pm**.

### Agenda

- Review of permutation testing.
- Using permutation tests to show causality.
- Bootstrapping.

## Review: Permutation testing

### Permutation testing (a.k.a. A/B testing)
- Given two samples, are they drawn from the same population?
- We can use permutation tests to answer questions like:
    - "Do smoking moms and nonsmoking moms have babies that weigh the same?"
    - "Were COVID-19 rates the same in Republican states and Democratic states?"
    - **More generally:** are *these things* like *those things*?

In [ ]:
show_permutation_testing_slides()

## Example: Deflategate 🏈

### Did the New England Patriots cheat?

<center><img width="40%" src="./data/deflate.jpg"></center>

- On January 18, 2015, the New England Patriots played the Indianapolis Colts for a spot in the Super Bowl.
- The Patriots won, 45-7. They went on to win the Super Bowl.
- After the game, it was alleged that the Patriots intentionally deflated footballs, making them easier to catch.

### Background

- Each team brings 12 footballs to the game. Teams use their own footballs while on offense.
- NFL rules stipulate that **each ball must be inflated to between 12.5 and 13.5 pounds per square inch (psi)**.
- Before the game, officials found that all of the Patriots' footballs were at about 12.5 psi, and that all of the Colts' footballs were at about 13.0 psi.
    - This pre-game data was not written down.
- In the second quarter, the Colts intercepted a Patriots ball and notified officials that it felt under-inflated.
- At halftime, two officials measured the pressures of as many of the 24 footballs as they could.
    - They ran out of time before they could finish.
- Note that the relevant quantity is the **change in pressure** from the start of the game to the halftime.
    - The Patriot's balls *started* at a lower psi (which is not an issue on its own).
    - The allegations were that the Patriots **deflated** their balls, during the game.

In [ ]:
footballs = bpd.read_csv('data/footballs.csv')
footballs

There are only 15 rows (11 for Patriots footballs, 4 for Colts footballs) since the officials weren't able to record the pressures of every ball.

### The question

- Did the Patriots' footballs drop in pressure more than the Colts'?
    - We want to test whether two samples came from the same distribution – this calls for a permutation test.
- **Null hypothesis**: The drop in pressures for both teams came from the same distribution.
    - By chance, the Patriots' footballs deflated more.
- **Alternative hypothesis**: No, the Patriots' footballs deflated more than one would expect due to random chance alone.

### The test statistic

Similar to the baby weights example, our test statistic will be the difference between the teams' average pressure drops.

In [ ]:
means = footballs.groupby('Team').mean().get('PressureDrop')
means

In [ ]:
# The observed statistic
observed_difference = means.loc['Patriots'] - means.loc['Colts']
observed_difference

The average pressure drop for the Patriots was 0.73 psi more than the Colts.

### Permutation test

- We'll run a permutation test to see if this is a significant difference.
- Remember, try it out once before iterating!

In [ ]:
# Keep only one column of labels and one column of numerical data
footballs = footballs.get(['Team', 'PressureDrop'])
footballs

In [ ]:
# Shuffle one column 
# We chose to shuffle the numerical data (pressure drops), but we could have shuffled the group labels (team names) instead
shuffled_drops = np.random.permutation(footballs.get('PressureDrop'))
shuffled_drops

In [ ]:
# Add the shuffled column back to the DataFrame
shuffled = footballs.assign(ShuffledPressureDrop=shuffled_drops)
shuffled

In [ ]:
# Calculate the difference in group means for the shuffled data
team_means = shuffled.groupby('Team').mean().get('ShuffledPressureDrop')
team_means

In [ ]:
# Difference in group means (Patriots minus Colts) for these randomly created groups
team_means.loc['Patriots'] - team_means.loc['Colts']

### Permutation test

- Repeat the process many times by wrapping it inside a **for** loop.
- Keep track of the difference in group means in an array, appending each time.
- Optionally, create a function to calculate the difference in group means.

In [ ]:
def difference_in_mean_pressure_drops(pressures_df):
    team_means = pressures_df.groupby('Team').mean().get('ShuffledPressureDrop')
    return team_means.loc['Patriots'] - team_means.loc['Colts']

In [ ]:
n_repetitions = 5000 # The dataset is much smaller than in the baby weights example, so a larger number of repetitions will still run quickly

differences = np.array([])
for i in np.arange(n_repetitions):
    # Step 1: Shuffle the pressure drops
    shuffled_drops = np.random.permutation(footballs.get('PressureDrop'))
    
    # Step 2: Put them in a DataFrame
    shuffled = footballs.assign(
        ShuffledPressureDrop=shuffled_drops
    )
    
    # Step 3: Compute the difference in group means and add the result to the differences array
    difference = difference_in_mean_pressure_drops(shuffled)

    differences = np.append(differences, difference)

### Conclusion

In [ ]:
bpd.DataFrame().assign(SimulatedDifferenceInMeans=differences).plot(kind='hist', bins=20, density=True, ec='w', figsize=(10, 5))
plt.axvline(observed_difference, color='red', label='observed difference in means')
plt.legend();

- It doesn't look good for the Patriots. What is the p-value?
    - Recall, the p-value is the **probability of seeing a result as or more extreme than the observation under the null hypothesis**.
    - In this case, that's the probability of the difference in mean pressure drops being greater than or equal to 0.7335.

In [ ]:
observed_difference

In [ ]:
# Calculating the p-value
np.count_nonzero(differences >= observed_difference) / n_repetitions

This p-value is low enough to consider this result to be highly statistically significant ($p<0.01$).

### Caution! ⚠️

- We conclude that it is unlikely that the difference in mean pressure drop is due to chance alone.
- But this doesn't establish *causation*.
- That is, we can't conclude that the Patriots **intentionally** deflated their footballs.
- This was an *observational* study; to establish causation, we'd need an RCT (Randomized Controlled Trial).

### Aftermath

- Quote from an investigative report commissioned by the NFL:

> “[T]he average pressure drop of the Patriots game balls exceeded the average pressure drop of the Colts balls by 0.45 to 1.02 psi, depending on various possible assumptions regarding the gauges used, and assuming an initial pressure of 12.5 psi for the Patriots balls and 13.0 for the Colts balls.”

- Many different methods were used to determine whether the drop in pressures were due to chance, including physics. 
    - We computed an observed difference of 0.7335, which is in line with the findings of the report. 
- In the end, Tom Brady (quarterback for the Patriots at the time) was suspended 4 games and the team was fined $1 million dollars.
- The [Deflategate Wikipedia article](https://en.wikipedia.org/wiki/Deflategate) is extremely thorough, give it a read if you're curious!

## Causality example: chronic back pain

### Causality and permutation tests

- Permutation tests can be used to establish **causality** in a randomized control trial!
- If the only difference between two groups is that one was given the treatment, and there is a statistically significant difference between the two groups, then we can conclude the treatment had some effect.

### Using Botulinum toxin A (Botox) to treat lower back pain


> [Botulinum neurotoxins (BoNTs) are the most potent toxins known.](https://febs.onlinelibrary.wiley.com/doi/10.1002/1873-3468.13446)

- Botox is commonly used for treating muscle disorders, migraines, and for cosmetic purposes.
- A randomized controlled trial examined the use of Botox in the treatment of lower back pain.
    - 31 patients with pain were randomly assigned to control and treatment groups.
    - The control group received a placebo (saline injection).
        - Placebos are used when we don't want individuals to know which group they are in.
    - The treatment group received Botox.
    - After eight weeks, the number of people who experienced relief in both groups was counted.

### The data

- 1 means "experienced relief".
- 0 means "no relief".

In [ ]:
back = bpd.read_csv('data/bta.csv')
back

In [ ]:
back.groupby('Group').count()

### The results

In [ ]:
# This evaluates to the proportion experiencing relief in each group
back.groupby('Group').mean()

- 60% of the treatment group experienced relief, compared to 12.5% of the control group.
- But what if the people in the treatment group would have gotten better without the treatment, by chance?
    - If this were the case, then the treatment would look like it had an impact even if it didn't.
    - To account for this possibility, we should conduct a hypothesis test.

### A permutation test

- Here, we have two numerical samples – the results for the control group, and the results for the treatment group.
- **Null hypothesis**: Results for both groups come from the same distribution. 
    - In other words, Botox does not do anything different than saline, and the results we saw are due to chance. 
- **Alternative hypothesis**: More people in the treatment group experience relief.
    - In other words, Botox helped with relief more than saline.
- **Test statistic**: difference in proportion experiencing relief.

In [ ]:
# This will work whether the column is 'Result' or 'ShuffledResult'
def difference_in_relief(df, column):
    grouped = df.groupby('Group').mean().get(column)
    return grouped.loc['Treatment'] - grouped.loc['Control']

In [ ]:
observed_difference = difference_in_relief(back, 'Result')
observed_difference

### Shuffling

In [ ]:
shuffled_results = np.random.permutation(back.get('Result'))
shuffled = back.assign(ShuffledResult=shuffled_results)
shuffled

- The above DataFrame contains the results of one "shuffling" of the data.
- This is equivalent to taking all participants in the trial and randomly assigning 16 to the control group and 15 to the treatment group.
    - The same effect could be achieved by shuffling the `'Group'` column instead.

### The simulation

In [ ]:
np.random.seed(4242) # Magic to ensure that we get the same results every time this code is run. 

n_repetitions = 500
differences = np.array([])

for i in np.arange(n_repetitions):
    # Step 1: Shuffle the results
    shuffled_results = np.random.permutation(back.get('Result'))
    
    # Step 2: Put the shuffled results in a DataFrame
    shuffled = back.assign(ShuffledResult=shuffled_results)
    
    # Step 3: Compute the difference in group proportions and add the result to the differences array
    difference = difference_in_relief(shuffled, 'ShuffledResult')
    
    differences = np.append(differences, difference)

### Visualization

In [ ]:
bpd.DataFrame().assign(DifferenceInMeans=differences).plot(kind='hist', bins=9, density=True, ec='w', figsize=(10, 5));
plt.axvline(observed_difference, color='red', label='observed difference')
plt.legend();

The p-value is the probability of seeing a test statistic equal to the observed test statistic or more extreme in the direction of the alternative hypothesis, under the null hypothesis.

In [ ]:
# p-value
np.count_nonzero(differences >= observed_difference) / n_repetitions

This is saying that the probability of seeing results this different in the two groups due to chance alone is vanishingly small (close to 0).

### Conclusion

- We reject the null hypothesis with a high degree of confidence.
- This is evidence that the treatment **caused** improvement.
    - **Only because** this was a **randomized controlled trial**.
    - In earlier examples (e.g. birth weights of babies from smoking moms and nonsmoking moms), we could not establish causality because there could have been other differences between the two groups.
- Read more about this example in [CIT 12.2](https://inferentialthinking.com/chapters/12/2/Causality.html?highlight=randomized%20control#potential-outcomes).

## Bootstrapping 🥾

### City of San Diego employee salary data

All City of San Diego employee salary data [is public](https://publicpay.ca.gov/Reports/Cities/City.aspx?entityid=405&year=2020&rpt=1). We are using the latest available data.

In [ ]:
population = bpd.read_csv('data/2020_salaries.csv')
population

When you load in a dataset that has so many columns that you can't see them all, it's a good idea to look at the column names.

In [ ]:
population.columns

### We only need the total wages...

In [ ]:
population = population.get(['TotalWages'])
population

In [ ]:
population.plot(kind='hist', bins=np.arange(0, 325000, 10000), density=True, ec='w', figsize=(10, 5));

### Discussion Question

Consider the question

> What is the median salary of all San Diego city employees?

What is the right tool to answer this question?

- A. Standard hypothesis testing
- B. Permutation testing
- C. Either of the above
- D. None of the above

### To answer, go to [menti.com](https://menti.com) and enter the code 9199 2833.

### The median salary

- We can use `.median()` to find the median salary of all city employees.
- This is **not** a random quantity.

In [ ]:
population_median = population.get('TotalWages').median()
population_median

### But now...

- ...suppose we don't have access to this entire dataset.
- In practice, it is costly and time-consuming to survey **all** 12,000+ employees.
    - More generally, we can't expect to survey all members of the population we care about.
- Instead, we gather salaries for a random sample of, say, 500 people.
- Hopefully, the median of the sample is close to the median of the population.

### In the language of statistics...

- The full DataFrame of salaries is the **population**.
- We observe a **sample** of 500 salaries from the population.
- We want to determine the **population median (a parameter)**, but we don't have the whole population, so instead we use the **sample median (a statistic) as an estimate**.
- Hopefully the sample median is close to the population median.

### The sample median

- Let's survey 500 employees at random.
- We can use `.sample()`:

In [ ]:
np.random.seed(23) # Magic to ensure that we get the same results every time this code is run

# Take a sample of size 500
my_sample = population.sample(500)
my_sample

We won't reassign `my_sample` at any point in this notebook, so it will always refer to this particular sample.

In [ ]:
# Compute the sample median
sample_median = my_sample.get('TotalWages').median()
sample_median

### How confident are we that this is a good estimate?

- Our estimate depended on a random sample.
- If our sample was different, our estimate may have been different, too.
- **How different could our estimate have been?**
- Our confidence in the estimate depends on the answer to this question.

### The sample median is random

- The sample median is a random number.
- It comes from some distribution, which we don't know.
- How different could our estimate have been, if we drew a different sample?
    - "Narrow" distribution $\Rightarrow$ not too different.
    - "Wide" distribution $\Rightarrow$ quite different.
- **What is the distribution of the sample median?**

### An impractical approach

- One idea: repeatedly collect random samples of 500 **from the population** and compute its median.
    - This is what we did in Lecture 14 to compute an empirical distribution of the sample mean of flight delays.
- The next two cells visualize the process of repeatedly collecting a sample and computing its median.

In [ ]:
%%capture
anim, sample_medians = sampling_animation(population);

In [ ]:
HTML(anim.to_jshtml())

### Visualize the empirical distribution

- We can plot the empirical distribution of the sample median with a histogram.
- This is an approximation of the true distribution of the sample median, using 246 samples.

In [ ]:
bpd.DataFrame().assign(SampleMedians=sample_medians) \
               .plot(kind='hist', density=True, bins=np.arange(60000, 85000, 1000), ec='w', figsize=(10, 5));

### The problem

- Drawing new samples like this is impractical.
    - If we were able to do this, why not just collect more data in the first place?
- Often, we can't ask for new samples from the population.
- **Key insight:** our original sample, `my_sample`, looks a lot like the population.
    - Their distributions are similar.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
bins=np.arange(10_000, 300_000, 10_000)
population.plot(kind='hist', y='TotalWages', ax=ax, density=True, alpha=.75, bins=bins, ec='w')
my_sample.plot(kind='hist', y='TotalWages', ax=ax, density=True, alpha=.75, bins=bins, ec='w')
plt.legend(['Population', 'My Sample']);

Note that unlike the previous histogram we saw, this is depicting the distribution of the population and of one particular sample (`my_sample`), **not** the distribution of sample medians for 246 samples.

### The bootstrap

- **Radical new idea:** What about sampling from the sample?
    - The sample itself looks like the population.
    - So, resampling from the sample is like sampling from the population.
    - The act of resampling from a sample is called **bootstrapping** or "**the bootstrap**" method.

- In our case specifically:
    - We have a sample of 500 salaries.
    - We want another sample of 500 salaries, but we can't draw from the population.
    - However, the original sample looks like the population.
    - So, let's just **resample from the sample!**

In [ ]:
show_bootstrapping_slides()

### Resampling with replacement

When bootstrapping, we resample **with** replacement. Why? 🤔

### Resampling with replacement

- Our goal when bootstrapping is to create a sample of the same size as our original sample.
- If we were to resample without replacement $n$ times from an original sample of size $n$, our resample would look exactly the same as the original sample.
    - For instance, if we sample 5 elements without replacement from `['A', 'B', 'C', 'D', 'E']`, our sample will contain the same 5 characters, just in a different order.
- So, we need to sample **with replacement** to ensure that our resamples can be different from the original sample.

### Running the bootstrap

- We can simulate the act of collecting new samples by **sampling with replacement from our original sample, `my_sample`**.

In [ ]:
# Note that the population DataFrame doesn't appear anywhere here!

n_resamples = 5000
boot_medians = np.array([])

for i in range(n_resamples):
    
    # Resample from my_sample WITH REPLACEMENT
    resample = my_sample.sample(500, replace=True)
    
    # Compute the median
    median = resample.get('TotalWages').median()
    
    # Store it in our array of medians
    boot_medians = np.append(boot_medians, median)

In [ ]:
boot_medians

### Bootstrap distribution of the sample median

In [ ]:
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist', density=True, bins=np.arange(60000, 85000, 1000), ec='w', figsize=(10, 5))
plt.scatter(population_median, 0.000004, color='orange', s=100, label='population median').set_zorder(2)
plt.legend();

- The population median (orange dot) is near the middle.
    - **In reality, we'd never get to see this!**

### Bootstrap rules of thumb

- The bootstrap is an awesome tool:
    - We only had to collect a single sample from the population to get the (approximate) distribution of the sample median.
- But it has limitations:
    - It is not good for sensitive statistics, like the max or min.
    - It requires the sample to be a good approximation of population.
        - If our original sample was not representative of the population, the resulting bootstrapped samples will also not be representative of the population.

### Example: estimating the max of a population

Suppose we want to estimate the maximum salary of all San Diego city employees, given just a single sample `my_sample`.

In [ ]:
# The true maximum salary in the population
population_max = population.get('TotalWages').max()
population_max

### Running the bootstrap

- We want to estimate the maximum number in the population.
- Our estimate will be the max in the sample.
- We run the bootstrap:

In [ ]:
n_resamples = 5000
boot_maxes = np.array([])

for i in range(n_resamples):

    resample = my_sample.sample(my_sample.shape[0], replace=True)
    
    boot_max = resample.get('TotalWages').max()
    
    boot_maxes = np.append(boot_maxes, boot_max)

In [ ]:
boot_maxes

### Visualize

The bootstrap distribution doesn't surround the population maximum (orange dot) of 320699. Why not? 🤔

In [ ]:
bpd.DataFrame().assign(BootstrapMax=boot_maxes).plot(kind='hist', 
                                                     density=True, 
                                                     bins=np.arange(180000, 325000, 20000), 
                                                     ec='w',
                                                     figsize=(10, 5))
plt.scatter(population_max, 0.0000008, color='orange', s=100, label='population max')
plt.legend();

### The answer

- The largest value in our original sample was 257062. 
- Therefore, the largest value in any bootstrapped sample is at most 257062.
- Generally, the bootstrap works better for measures of central tendency or variation (means, medians, variances) than it does for extremas (maxes and mins).
    - If we instead repeatedly sampled from the population directly, there would be more possibilities for the largest possible value in each sample, and so the distribution of the sample max would not have the "gaps" in it that the distribution on the previous slide does.

In [ ]:
my_sample.get('TotalWages').max()

## Summary

### Goal

- Given a single sample, we want to estimate population parameter (e.g. the population median). 
- To do this, we compute a relevant statistic on our sample (e.g. the population statistic).
- However, this estimate is random, because our sample is random, and so it could have looked different.
- In order to know whether our estimate is reliable, we'd like to look at its distribution.
- Unfortunately, determining the distribution of our estimate would require us to collect more samples from our population, which we can't do in practice.

### Key idea

- **The distribution of a sample looks a lot like the distribution of the population it was drawn from.**
- So, we can repeatedly **resample** from our original sample, and compute the estimate (e.g. median) on each of these resamples to create a **bootstrapped** distribution of our estimate.
    - Resampling from a sample is called **bootstrapping**.
- This bootstrapped distribution will look a lot like the true distribution of the estimate.

### Next steps

- We just learned how to approximate the distribution of a sample statistic.
- This means that we now have a sense of how much our estimates can vary.
- But if someone asks, "what's your best guess of the population parameter?" we're not going to give them our bootstrapped distribution. Instead, we may want to give them a range, or **interval**, of likely values.
- Next time, we'll formalize the notion of a confidence interval, and will talk about the idea of percentiles along the way.